## Importing libraries we will use
### Pandas for dataframes, numpy for some plotting, web scraping tools, file creating tools

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
from itertools import islice
import re
import docx

## This will help me clean up HTML text

In [2]:
import re
CLEANR = re.compile('<.*?>') 
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

We will be scraping the following forum thread, https://forums.somethingawful.com/showthread.phpthreadid=3550307&userid=0&perpage=40&pagenumber=1
, it's a choose your own adventure game where the creator posits a scenario, people vote on the action they want to take 
and the author continues the story based on that

The game has been running for over 9 years, and hundreds of people have been involved in the voting. The forum lacks both a good search functionality and statistics regarding the posting.

Players and the writer have asked me for different stats, so this analysis will be immediately used by real people other than me

In [3]:
def rindex(lst, value):
    lst.reverse()
    i = lst.index(value)
    lst.reverse()
    return len(lst) - i - 1
def nth_index(iterable, value, n):
    matches = (idx for idx, val in enumerate(iterable) if val == value)
    return next(islice(matches, n-1, n), None)
def find_nth_occurrence(l, value, n):
    indices = [index for index, item in enumerate(l)
               if item == value]
    if len(indices) < n:
        return -1
    return indices[n - 1]

In [4]:
# saving today's date
now = datetime.now()

In order to automate the scraping we need to know how many pages there are,as it's continuously increasing
We will use the "last page" link in the first page to obtain that

In [5]:
LitThread="https://forums.somethingawful.com/showthread.php?threadid="
ParadiseLost="https://forums.somethingawful.com/showthread.php?threadid=3550307&userid=0&perpage=40&pagenumber="
response=requests.get("https://forums.somethingawful.com/showthread.php?threadid=3550307&userid=0&perpage=40&pagenumber=1")
MainPage=BeautifulSoup(response.text)
LastpageLink=MainPage.find("a",title="Last page")
LastPage=str(LastpageLink)
LastPageNumber=int(LastPage[LastPage.rfind("=")+13:LastPage.rfind(" ")])
print(LastPageNumber)

7749


Scraping all ~7700 pages would take too long, so I'll use a smaller sample.
To scrape the whole thing all we need to change is the range in the for to "LastPageNumber"
Scraping the "threadlength" pages of the thread

At first I was saving all pages in a list, and then reiterating through it to get the data, then i realized it wasn't as efficient as it could be

In [6]:
#legacy
#Pages=[]
#print(Pages)
#threadlength=1
#for i in range(threadlength):
# Response=requests.get("https://forums.somethingawful.com/showthread.php?threadid=3550307&userid=0&perpage=40&pagenumber="+str(j))
# pageTemp=BeautifulSoup(Response.text)
# Pages.append(pageTemp)
# print(i)

In [7]:
#legacy
#Creating a Dictionary for every posts, it includes the username,the date on which the post was realized and the raw content
#PostsDictionary={"User_Name":[],"Date":[],"Post_Content":[]}
#postsperpage=42
#for pagenumber in range(len(Pages)):
 #   print(pagenumber)
 #   for i in range(postsperpage):
   #     username=str(Pages[pagenumber].find_all("td",class_="userinfo")[i].find("dt"))
    #    usernameclean=username[username.find(">")+1:username.find("<",1)]
    #    if usernameclean!="Adbot":
     #       PostsDictionary["User_Name"].append(usernameclean)
      #      date=str(Pages[pagenumber].find_all("td",class_="postdate")[i])
      #      dateofpost=pd.to_datetime(str(date[date.rfind("a>")+3:date.rfind("a>")+21]))
      #      PostsDictionary["Date"].append(dateofpost)
       #     text=str(Pages[pagenumber].find_all("td",class_="postbody")[i])
       #     PostsDictionary["Post_Content"].append(text)
           #  text=str(pageTemp.find_all("td",class_="postbody")[k])
            # PostsDictionary=["Post_Content"].append(usernameclean)


So what I ended up doing was scraping each page and obtainign the data, that way I don't need to save the pages and only iterate through all of them once

Scraping each page
Creating a Dictionary for every posts, it includes 

a)Username

b)Date on which the post was realized

c)Text content

In [8]:
PostsDictionary={"User_Name":[],"Date":[],"Post_Content":[],"Post_Words":[]}
postsperpage=42 #We know there are 40 user posts in each page, and 2 ads which are counted as posts
for i in range(626):
    j=i+1
    print("Scraping page "+str(j))
    Response=requests.get("https://forums.somethingawful.com/showthread.php?threadid=3643994&userid=0&perpage=40&pagenumber="+str(j))
    pageTemp=BeautifulSoup(Response.text)
    for k in range(postsperpage):
        username=str(pageTemp.find_all("td",class_="userinfo")[k].find("dt"))
        usernameclean=username[username.find(">")+1:username.find("<",1)]
        if usernameclean!="Adbot": #name of the user that posts the ads previously mentioned
            PostsDictionary["User_Name"].append(usernameclean)
            date=str(pageTemp.find_all("td",class_="postdate")[k])
            dateofpost=pd.to_datetime(str(date[date.rfind("a>")+3:date.rfind("a>")+21]))
            PostsDictionary["Date"].append(dateofpost)
            text=str(pageTemp.find_all("td",class_="postbody")[k])
            text=cleanhtml(text)
            text=text.replace("\n","")
            PostsDictionary["Post_Content"].append(text)
            PostsDictionary["Post_Words"].append(len(text.split()))

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50
Scraping page 51
Scraping page 52
Scraping page 53
Scraping page 54
Scraping page 55
Scraping page 56
Scraping page 57
Scraping page 58
Scraping page 59
Scrapi

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [ ]:
Response=requests.get(LitThread+str(j))
pageTemp=BeautifulSoup(Response.text)
#pageTemp.find_all("td",class_="userinfo")
pageTemp

Transforming into a dataframe for ease of use

In [ ]:
DFposts=pd.DataFrame(PostsDictionary)
DFposts

## Creating a list of unique posters

In [ ]:
UniqueUsersList=[]
for item in PostsDictionary["User_Name"]: 
    if item not in UniqueUsersList: 
        UniqueUsersList.append(item) 

In [ ]:
pd.Series(PostsDictionary["User_Name"]).value_counts().plot(kind="bar")

## Now I'm going to create aggregated data for each user, not on a post by post basis

Includes

a)User name

b)Date on which they FIRST posted

c)Number of times they've posted

d)Posts per day. Begin date: first post's date. End date: Latest date on which somebody posted.

In [ ]:
UserData={"User_Name":[],"First_Post":[],"Total_Posts":[],"Post_Rate":[]}

In [ ]:
#Filling that dictionary with the names"
UserData["User_Name"].clear()
for item in PostsDictionary["User_Name"]: 
    if item not in UniqueUsersList:
        UniqueUsersList.append(item) 
for user in UniqueUsersList:
    UserData["User_Name"].append(user)

In [ ]:
#Filling that dictionary with the date of their first post
for user in UserData["User_Name"]: 
   firstappearance=PostsDictionary["User_Name"].index(user) #finding the first apppearance of a poster in the PostsDictionary
   UserData["First_Post"].append(PostsDictionary["Date"][firstappearance])

In [ ]:
#Filling the dictionary with total posts
totalposts=pd.Series(PostsDictionary["User_Name"]).value_counts()
for user in UserData["User_Name"]:
    UserData["Total_Posts"].append(totalposts[user])

In [ ]:
LatestPostDate=max(PostsDictionary["Date"])

In [ ]:
UserData["Post_Rate"].clear()
for i in range(len(UserData["User_Name"])):
    TimeSincePost=str((LatestPostDate-UserData["First_Post"][i]))
    DaysSincePost=int(TimeSincePost[0:TimeSincePost.find("days")])
    if DaysSincePost==0:
        DaysSincePost=1
        UserData["Post_Rate"].append(UserData["Total_Posts"][i]/DaysSincePost)
    else:
        UserData["Post_Rate"].append(UserData["Total_Posts"][i]/DaysSincePost)

### Transforming the userdata dictionary into a Datrame

In [ ]:
DFusers=pd.DataFrame(UserData)

#### Adding a new column wiht the totla and average amount of words per post for each user

In [ ]:
Total_Words=[]
for user in UniqueUsersList:
    mask1=DFposts["User_Name"]==user
    totalwords=DFposts["Post_Words"][mask1].sum()
    Total_Words.append(totalwords)
DFusers["Total_Words"]=Total_Words
DFusers["Average_Words"]=DFusers["Total_Words"]/DFusers["Total_Posts"]

In [ ]:
DFusers[DFusers["Total_Posts"]>10].sort_values("Total_Words",ascending=[False])


### Categorizing users based on the "effort" of their posts

In [ ]:
PosterType=[]
LowEffort=20
MidEffort=40
i=-1
for user in UniqueUsersList:
    i=i+1
    if DFusers[DFusers["User_Name"]==user]["Average_Words"][i]<LowEffort:
        PosterType.append("Low effort poster")
    else:
            if DFusers[DFusers["User_Name"]==user]["Average_Words"][i]<MidEffort:
                PosterType.append("Low effort poster")
            else:
                    PosterType.append("High effort poster")    

In [ ]:
DFusers["Poster_Type"]=PosterType

## Plotting total posts and total words, we can see an extreme outlier in the writer, as not only he has more posts, but way more words as well. This makes sense as he is the one writing the story

In [ ]:
plt.scatter(DFusers["Total_Posts"],DFusers["Total_Words"])

### Here we are calculating the largest time a poster spent without posting in the thread. This only compares within the period he was active, that is "Start date"=FirstPostDate, "EndDate"=LastPostDate, so someone with only one post will have a gap of 0

In [ ]:
gaps=[]
for user in UniqueUsersList:
  gap=timedelta()
  for i in range(UserData["Total_Posts"][UserData["User_Name"].index(user)]-1):
      gaptemp=PostsDictionary["Date"][find_nth_occurrence(PostsDictionary["User_Name"], user ,i+2)]-PostsDictionary["Date"][find_nth_occurrence(PostsDictionary["User_Name"], user ,i+1)]
      if gaptemp>gap:
        gap=gaptemp
        
  # usergap=PostsDictionary["Date"][rindex(PostsDictionary["User_Name"],user)]-PostsDictionary["Date"][PostsDictionary["User_Name"].index(user)]
#  print(user + "'s largest gap is " + str(gap.days)+" days")


### As I mentioned before the forum lacks a good search functionality. And as the game has been running for so many years, and has over 7000 pages of posts it's hard to find what you are looking for.

### The creator of the game requested I try to create a unified word document with his posts for ease of search

### Creating a list of the clean text of all of his posts, I'm scraping a link that specifically only includes his post, it's more efficient than scraping all the thread and using an "if" to see if it should be saved

In [ ]:
OPPosts=[]
for i in range(10):
    j=i+1
    print("Scraping page "+str(j))
    Response=requests.get("https://forums.somethingawful.com/showthread.php?threadid=3550307&userid=129241&perpage=40&pagenumber="+str(j))
    pageTemp=BeautifulSoup(Response.text)
    for k in range(postsperpage):
        username=str(pageTemp.find_all("td",class_="userinfo")[k].find("dt"))
        usernameclean=username[username.find(">")+1:username.find("<",1)]
        if usernameclean!="Adbot":
            text=str(pageTemp.find_all("td",class_="postbody")[k])
            text=text.replace("\n","")
            text=cleanhtml(text)
            OPPosts.append(text)

## Creating the document with the information, it can now be easily searched in Word

In [ ]:
Diogposts = docx.Document()
for i in range(len(OPPosts)):
    Diogposts.add_paragraph(OPPosts[i])
    Diogposts.add_paragraph("POST FINISHED")
    Diogposts.add_paragraph(" ")
    Diogposts.add_paragraph(" ")
    Diogposts.add_paragraph(" ")
    Diogposts.add_paragraph("POST BEGINS")
Diogposts.save(r"C:\Users\Octavio Ciappina\Documents\Python\MBA\text.docx")

## exporting to excel for ease of use

In [ ]:
DFusers.to_excel("UserData.xlsx")
DFposts.to_excel("PostsData.xlsx") #this is slightly heavy for a excel file, specially if i scrape the full thread

In [16]:
postsperpage=42

In [68]:
def userScraping(thread):
    gap=timedelta()   
    response=requests.get(thread+"1")
    totalwords=0
    UserPosts={"User_Name":[],"Date":[],"Post_Content":[],"Post_Words":[]}
    MainPage=BeautifulSoup(response.text)
    LastpageLink=MainPage.find("a",title="Last page")
    LastPage=str(LastpageLink)
    LastPageNumber=int(LastPage[LastPage.rfind("=")+13:LastPage.rfind(" ")])
    for i in range(LastPageNumber-1):
        j=i+1
        print("Scraping page "+str(j))
        Response=requests.get(thread+str(j))
        pageTemp=BeautifulSoup(Response.text)
        for k in range(postsperpage):
            username=str(pageTemp.find_all("td",class_="userinfo")[k].find("dt"))
            usernameclean=username[username.find(">")+1:username.find("<",1)]
            if usernameclean!="Adbot": #name of the user that posts the ads previously mentioned
                UserPosts["User_Name"].append(usernameclean)
                date=str(pageTemp.find_all("td",class_="postdate")[k])
                dateofpost=pd.to_datetime(str(date[date.rfind("a>")+3:date.rfind("a>")+21]))
                UserPosts["Date"].append(dateofpost)
                text=str(pageTemp.find_all("td",class_="postbody")[k])
                text=cleanhtml(text)
                text=text.replace("\n","")
                UserPosts["Post_Content"].append(text)
                UserPosts["Post_Words"].append(len(text.split()))    
    for i in range(len(UserPosts["Date"])-1):
        gaptemp=UserPosts["Date"][i+1]-UserPosts["Date"][i]
        if gaptemp>gap:
            gap=gaptemp
    totalwords=sum(UserPosts["Post_Words"])
    averagewords=totalwords/len(UserPosts["Date"])
    timeactive=max(UserPosts["Date"])-UserPosts["Date"][0]
    daysactive=timeactive.days
    postrate=len(UserPosts["Date"])/daysactive
    return(usernameclean + " has "+str(len(UserPosts["Date"]))+" posts. His posts rate is "+str(postrate)+" posts per day he has been active. The longest he has not posted is "+str(gap.days)+" days. He has written a total of "+str(totalwords)+" words. Average words per post is "+ str(averagewords)+".")        

In [69]:
userScraping("https://forums.somethingawful.com/showthread.php?threadid=3550307&userid=99457&perpage=40&pagenumber=")

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50
Scraping page 51
Scraping page 52
Scraping page 53
Scraping page 54
Scraping page 55
Scraping page 56
Scraping page 57
Scraping page 58
Scraping page 59
Scrapi

'Swedish Thaumocracy has 2400 posts. His posts rate is 0.7970773829292593 posts per day he has been active. The longest he has not posted is 161 days. He has written a total of 200258 words. Average words per post is 83.44083333333333.'

In [ ]:
for i in range(60):
    j=i+1
    print("Scraping page "+str(j))
    Response=requests.get("https://forums.somethingawful.com/showthread.php?threadid=3643994&userid=0&perpage=40&pagenumber="+str(j))
    pageTemp=BeautifulSoup(Response.text)
    for k in range(postsperpage):
        username=str(pageTemp.find_all("td",class_="userinfo")[k].find("dt"))
        usernameclean=username[username.find(">")+1:username.find("<",1)]
        if usernameclean!="Adbot": #name of the user that posts the ads previously mentioned
            PostsDictionary["User_Name"].append(usernameclean)
            date=str(pageTemp.find_all("td",class_="postdate")[k])
            dateofpost=pd.to_datetime(str(date[date.rfind("a>")+3:date.rfind("a>")+21]))
            PostsDictionary["Date"].append(dateofpost)
            text=str(pageTemp.find_all("td",class_="postbody")[k])
            text=cleanhtml(text)
            text=text.replace("\n","")
            PostsDictionary["Post_Content"].append(text)
            PostsDictionary["Post_Words"].append(len(text.split()))
            